In [25]:
import pandas as pd

import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots


pio.orca.config.use_xvfb = True




# Read in latest data

Makefile has tasks for pulling from CovidTracking.com api

Need to download current season file from https://gis.cdc.gov/grasp/fluview/mortality.html and run make task to

In [26]:
weekOrder = [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,  1,  2,  3,  4,
        5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39]

usDaily = pd.read_json('file:///home/jovyan/work/datasets/CovidTrackingUsDaily.json')

usFluHistorical = pd.read_csv('file:///home/jovyan/work/datasets/National_Historical_Flu_Mortality.csv', thousands=',')
usFluCurrent = pd.read_csv('file:///home/jovyan/work/datasets/National_Current_Flu_Mortality.csv', thousands=',')
usFlu = pd.concat([usFluHistorical, usFluCurrent])

usFlu['SEASON'] = usFlu['SEASON'].astype('category')
usFlu['WEEK'] = usFlu['WEEK'].astype('category')

# Aggregate the CovidTracking data by week

In [3]:
usDaily['date'] = pd.to_datetime(usDaily['date'], format='%Y%m%d')

usDaily['dayOfYear'] = usDaily['date'].dt.strftime('%j')
usDaily['dayOfYear'] = usDaily['dayOfYear'].astype('int')


usDaily['week'] = pd.to_datetime(usDaily['date'] - pd.to_timedelta(7, unit='d'))
usDaily['week'] = usDaily['week'].dt.strftime('%W')

usDaily['week'] = usDaily['week'].astype('category')
col = usDaily.pop("week")
col = col.astype('int') + 2
usDaily.insert(0, col.name, col)

usDailyByWeek = usDaily.groupby(['week']).agg({'deathIncrease':'sum', 'hospitalizedIncrease':'sum', 'positiveIncrease':'sum', 'negativeIncrease':'sum', 'totalTestResultsIncrease':'sum'}).reset_index()

usDailyCurrentWeek = usDailyByWeek.tail(1)
usDailyByWeekExcludingCurrent = usDailyByWeek[:-1]

merged = pd.merge(usFlu, usDailyByWeekExcludingCurrent, how='left', left_on='WEEK', right_on='week')
merged['WEEK'] = merged['WEEK'].astype('category')

usDailyCurrentWeek

,week,deathIncrease,hospitalizedIncrease,positiveIncrease,negativeIncrease,totalTestResultsIncrease
4,14,5886.0,19001.0,166694.0,625762.0,792456.0


# Checking out the severity of flu seasons

Plot of all the flu deaths for each season

In [30]:
flufig = px.line(usFlu, x='WEEK', y='NUM INFLUENZA DEATHS', color='SEASON')
flufig.update_layout(xaxis_type = 'category',     yaxis_title="Deaths", title_text="US: Weekly Deaths of All Flu's since 2013 vs COVID-19 deaths")

flufig.add_scatter(x=merged['WEEK'], y=merged['deathIncrease'], mode='lines', name="COVID-19", line=dict(color="darkgreen"))
flufig.add_scatter(x=usDailyCurrentWeek['week'], y=usDailyCurrentWeek['deathIncrease'], mode='markers', line=dict(color="darkgreen"), name="COVID-19 Incomplete Current Week")

flufig.update_xaxes(range=[40, 39])
flufig.show()
flufig.write_image('covid_images/US_All_Flu_Mortality_Comparison.png')

# Correlation of COVID data


 00-.19 "very weak"
 
.20-.39 "weak"

.40-.59 "moderate"

.60-.79 "strong"

.80-1.0 "very strong"

In [5]:
usDaily[['dayOfYear', 'deathIncrease', 'hospitalizedIncrease', 'negativeIncrease', 
         'positiveIncrease', 'totalTestResultsIncrease']].corr(method ='pearson')

,dayOfYear,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
dayOfYear,1.000000,0.831512,0.823052,0.911511,0.932444,0.923326
deathIncrease,0.831512,1.000000,0.734457,0.880992,0.951422,0.902473
hospitalizedIncrease,0.823052,0.734457,1.000000,0.797974,0.858992,0.816874
negativeIncrease,0.911511,0.880992,0.797974,1.000000,0.948169,0.997970
positiveIncrease,0.932444,0.951422,0.858992,0.948169,1.000000,0.966483
totalTestResultsIncrease,0.923326,0.902473,0.816874,0.997970,0.966483,1.000000


Pearon's R shows high statistical correlation across the board, most importantly everything is strongly correlated 
to total test results. Let's try combining a few of these metrics with totalTestResultIncrease to try and remove that correlation. I'll call these saturation metrics for lack of a better term and represent them as percents.

In [6]:
usDaily['deathSaturation'] = usDaily['deathIncrease']/usDaily['totalTestResultsIncrease'] * 100
usDaily['hospitalizedSaturation'] = usDaily['hospitalizedIncrease']/usDaily['totalTestResultsIncrease'] * 100
usDaily['negativeSaturation'] = usDaily['negativeIncrease']/usDaily['totalTestResultsIncrease'] * 100
usDaily['positiveSaturation'] = usDaily['positiveIncrease']/usDaily['totalTestResultsIncrease'] * 100

In [7]:
usDaily[['dayOfYear', 'deathSaturation', 'hospitalizedSaturation', 'negativeSaturation', 
         'positiveSaturation', 'totalTestResultsIncrease']].corr(method ='pearson')

,dayOfYear,deathSaturation,hospitalizedSaturation,negativeSaturation,positiveSaturation,totalTestResultsIncrease
dayOfYear,1.000000,0.517034,0.733180,-0.270949,0.270949,0.923326
deathSaturation,0.517034,1.000000,0.426608,-0.619635,0.619635,0.597174
hospitalizedSaturation,0.733180,0.426608,1.000000,-0.336718,0.336718,0.664044
negativeSaturation,-0.270949,-0.619635,-0.336718,1.000000,-1.000000,-0.303610
positiveSaturation,0.270949,0.619635,0.336718,-1.000000,1.000000,0.303610
totalTestResultsIncrease,0.923326,0.597174,0.664044,-0.303610,0.303610,1.000000


Positive Saturation and totalTestResultsIncrease are weakly correlated compared to positiveIncrease being very strongly correlated .

Need to analyze correlation between other pairs. 


- 00-.19 "very weak"
  - 

- .20-.39 "weak"
  - 

- .40-.59 "moderate"
  - 

- .60-.79 "strong"
  - 

- .80-1.0 "very strong"
  - 


# How saturated is the population with positive's over time

Active cases is a poor metric to measure spread since the number of tests are changing over time. It's possible that we could measure the spread of infection by looking at how saturated the total test population is with positive results overtime. This is expressed as a percentage.

The first week of data looks to have a lot of noise in it given the low volume of testing, so is removed in the last plot.

In [8]:
fig = make_subplots(rows=2, cols=2, subplot_titles=["Positive Tests", "Total Tests", "% Tests Positive", "% Test Positive Excluding First Week"])
fig.add_trace(go.Scatter(x=usDaily['date'], y=usDaily['positiveIncrease']), row=1, col=1)
fig.add_trace(go.Scatter(x=usDaily['date'], y=usDaily['totalTestResultsIncrease']), row=1, col=2)
fig.add_trace(go.Scatter(x=usDaily['date'], y=usDaily['positiveSaturation']), row=2, col=1)

droppedNoise = usDaily.head(-7)
fig.add_trace(go.Scatter(x=droppedNoise['date'], y=droppedNoise['positiveSaturation']), row=2, col=2)

fig.update_layout(height=600, width=900, title_text="Positive Tests Dependency On Total Test", showlegend=False)
fig.show()

fig.write_image('covid_images/US_COVID_Positive_Tests_Dependence.png')

In [ ]:
testsfig = go.Figure()
testsfig = px.scatter(droppedNoise, x='date', y='totalTestResultsIncrease')
testsfig.update_layout(title_text="US: Daily COVID Tests")
testsfig.show()

testsfig.write_image('covid_images/US_Daily_Total_COVID_Tests.png')

In [27]:
fig = go.Figure()
fig = px.box(usDaily, y="totalTestResultsIncrease")
fig.show()

## Fitting a line to positive saturation over time to predict possible first case

In [10]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.sandbox.regression.predstd import wls_prediction_std

regressionModelOfAllData = ols("positiveSaturation ~ dayOfYear", data=usDaily).fit()

regressionModelOfAllData.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     positiveSaturation   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     2.298
Date:                Sat, 04 Apr 2020   Prob (F-statistic):              0.140
Time:                        23:19:38   Log-Likelihood:                -99.704
No. Observations:                  31   AIC:                             203.4
Df Residuals:                      29   BIC:                             206.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.6700     10.084      0.166      0.870     -18.953      22.293
dayOfYear      0.1899      0.125      1.516      0.140      -0.066       0.446
==============================================================================
Omnibus:                       14.338   Durbin-Watson:                   2.038
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               15.073
Skew:                           1.352   Prob(JB):                     0.000533
Kurtosis:                       5.089   Cond. No.                         725.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Analysis of regression on Full Dataset

R-squared of 0.014 is abysmal. 

In [11]:
fig = go.Figure()
fig = px.box(usDaily, y="positiveSaturation")
fig.show()

In [12]:
#q1 = usDaily['positiveSaturation'].quantile(0.25)
#q3 = usDaily['positiveSaturation'].quantile(0.75)
#iqr = q3 - q1

#usDaily['positiveSaturationCorrected'] = usDaily['positiveSaturation'].clip( (q1 - 1.5 *iqr), (q3 + 1.5 * iqr) )

## Was not satisfied with the results of doing this statistically with zscores or IQR ...
# so throw out the first week of testing, based on it being over 2,000 tests/day

usDaily[['positiveSaturation', 'totalTestResultsIncrease']]

,positiveSaturation,totalTestResultsIncrease
0,15.599433,216463.0
1,23.560131,139596.0
2,24.021165,117742.0
3,25.843041,101122.0
4,23.217341,104030.0
5,18.914919,113503.0
6,21.774860,95647.0
7,17.255733,109071.0
8,17.408081,107295.0
9,17.184017,97806.0


In [13]:
fig = go.Figure()
fig = px.box(droppedNoise, y="positiveSaturation")
fig.show()

In [14]:
regressionModel = ols("positiveSaturation ~ dayOfYear", data=droppedNoise).fit()
print(regressionModel.params)
regressionModel.summary()

#posSat = -31.57 + 0.574 * dayOfYear

Intercept   -30.638514
dayOfYear     0.563472
dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     positiveSaturation   R-squared:                       0.687
Model:                            OLS   Adj. R-squared:                  0.673
Method:                 Least Squares   F-statistic:                     50.38
Date:                Sat, 04 Apr 2020   Prob (F-statistic):           3.13e-07
Time:                        23:19:39   Log-Likelihood:                -60.721
No. Observations:                  25   AIC:                             125.4
Df Residuals:                      23   BIC:                             127.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -30.6385      6.614     -4.633      0.000     -44.320     -16.957
dayOfYear      0.5635      0.079      7.098      0.000       0.399       0.728
==============================================================================
Omnibus:                        6.277   Durbin-Watson:                   1.611
Prob(Omnibus):                  0.043   Jarque-Bera (JB):                4.297
Skew:                          -0.937   Prob(JB):                        0.117
Kurtosis:                       3.785   Cond. No.                         963.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
regressionModel.predict({"dayOfYear": 55})

# R^2 still isn't satsifying ... need more data
# Rough estimate of 54th day of the year for the first positive test, this could be +/- 10 days (February 23rd)

0    0.352426
dtype: float64

In [16]:
#fig = plt.figure()

#sm.graphics.plot_regress_exog(regressionModel, "dayOfYear", fig=fig)


# predictor variable (x) and dependent variable (y)
x = droppedNoise['date']
y = droppedNoise['positiveSaturation']

# Retrieve our confidence interval values
# _ is a dummy variable since we don't actually use it for plotting but need it as a placeholder
# since wls_prediction_std(housing_model) returns 3 values
_, confidence_interval_lower, confidence_interval_upper = wls_prediction_std(regressionModel)



postestfig = go.Figure()
postestfig.update_layout(title_text="US: Percentage of Positive Tests (Slope " + str(round(regressionModel.params['dayOfYear'], 3)) + ", R Squared " + str(round(regressionModel.rsquared, 3)) + ")")




postestfig.add_scatter(x=x, y=droppedNoise['positiveSaturation'], mode='markers', name="% Positive Tets")
postestfig.add_scatter(x=x, y=regressionModel.fittedvalues, line=dict(color='green', width=1), name="Regression")

postestfig.add_trace(go.Scatter(x=x, y=confidence_interval_upper, name="Upper Confidence", line = dict(color='orangered', width=1, dash='dash')))
postestfig.add_trace(go.Scatter(x=x, y=confidence_interval_lower, name="Lower Confidence", line = dict(color='red', width=1, dash='dash')))



postestfig.show()
postestfig.write_image('covid_images/US_Positive_Tests_Over_Time.png')



# Looking at some more interesting saturations

In [17]:
fig = make_subplots(rows=2, cols=2)
fig.add_trace(go.Scatter(x=droppedNoise['date'], y=droppedNoise['deathSaturation']), row=1, col=1)
fig.add_trace(go.Scatter(x=droppedNoise['date'], y=droppedNoise['hospitalizedSaturation']), row=1, col=2)
fig.add_trace(go.Scatter(x=droppedNoise['date'], y=droppedNoise['negativeSaturation']), row=2, col=1)
fig.add_trace(go.Scatter(x=droppedNoise['date'], y=droppedNoise['positiveSaturation']), row=2, col=2)

fig.update_layout(height=600, width=900, title_text="Exploration", showlegend=False)
fig.show()

In [23]:
import chart_studio
import chart_studio.plotly as csplot


if False:
    chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

    csplot.plot(postestfig, filename='US Percentage of Positive Tests', auto_open=True)
    csplot.plot(testsfig, filename='US Daily Total Test Results', auto_open=True)
    csplot.plot(flufig, filename='US Weekly COVID Deaths vs Flu', auto_open=True)

In [19]:
usDaily[['date', 'totalTestResultsIncrease']]

,date,totalTestResultsIncrease
0,2020-04-04,216463.0
1,2020-04-03,139596.0
2,2020-04-02,117742.0
3,2020-04-01,101122.0
4,2020-03-31,104030.0
5,2020-03-30,113503.0
6,2020-03-29,95647.0
7,2020-03-28,109071.0
8,2020-03-27,107295.0
9,2020-03-26,97806.0
